In [1]:
import pandas as pd

## Load Dataset

In [2]:
df = pd.read_csv('order_brush_order.csv', parse_dates=['event_time'])
df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [17]:
shopid_list = df['shopid'].unique()
len(shopid_list)

18770

## Solution

In [27]:
%%time
solution2 = {}
for shopid in shopid_list:
    solution2[shopid] = '0'

cnt_sol = 0
for shopid, df_sample in df.groupby(['shopid']):
    order_brushing_orderids = []
    df_sample = df_sample.sort_values('event_time')

    s = 0
    for i in range(len(df_sample)):
        while df_sample.iloc[i].event_time - df_sample.iloc[s].event_time > pd.Timedelta('1H'):
            s += 1
        
        df_in_hour = df_sample.iloc[s:i+1]
        
        num_orders = len(df_in_hour)
        num_uniq_buyers = len(df_in_hour['userid'].unique())

        if num_orders >= 3 * num_uniq_buyers:
          order_brushing_orderids.extend(list(df_in_hour['orderid']))
    
    order_brushing_orderids = list(set(order_brushing_orderids))
  
    if len(order_brushing_orderids) > 0:
        df_brush = df_sample[df_sample['orderid'].isin(order_brushing_orderids)]
        df_users_cnt = df_brush[['orderid', 'userid']].groupby('userid').count()
        df_users_cnt.columns = ['count']

        max_cnt = df_users_cnt['count'].max()

        sus_users = list(df_users_cnt[df_users_cnt['count'] == max_cnt].index)
        sus_users_str = '&'.join(map(str, sus_users))

        solution2[shopid] = sus_users_str

        cnt_sol += 1
        print(cnt_sol, shopid, sus_users_str)

1 10402 77819
2 10536 672345
3 42472 740844
4 42818 170385453
5 76934 190449497
6 425364 72914921
7 599533 264511
8 605561 181682008
9 722132 7670129
10 731606 75558350
11 769445 141006168
12 823357 188942105
13 1175477 122277324
14 1532569 181408876
15 2831130 15053804
16 3124091 214568881
17 3701311 80690628
18 4149345 212325226
19 4888564 143847348
20 6765647 740844
21 8996761 215382704
22 10199219 8405753
23 10206302 95058664
24 10498480 199416406
25 11342466 152292010
26 11474437 148215831
27 11612863 214546342
28 11809607 136680607
29 11849988 148215831
30 12078788 214588488
31 12537642 48412388
32 13735592 215424202
33 14184981 32594
34 14598427 128702876
35 15920950 211907762
36 16001939 205729485
37 16246487 214300427
38 16680589 102125116
39 16732896 214094512
40 17011899 115994773
41 17212658 107170776
42 17219984 154495684
43 18190345 89014205
44 18229789 192400689
45 19052728 2824209
46 20522747 170673735
47 22800308 99997899
48 27015534 188025647
49 27476241 174145893
50 

In [28]:
df_solution = pd.DataFrame({'shopid' : list(solution2.keys()),
                            'userid' : list(solution2.values())})
df_solution.head()

,shopid,userid
0,93950878,0
1,156423439,0
2,173699291,0
3,63674025,0
4,127249066,0


In [29]:
df_solution.to_csv('solution7.csv', index=False)

In [30]:
len(df_solution[df_solution['userid'] != '0'])

264